<a href="https://colab.research.google.com/github/humayraR/fact-ai_Repl/blob/main/SynthDECAF_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# save pipe.pkl to output data folder
# !cp <output-file-name> /content/drive/MyDrive/GitCode


In [2]:
# !pip install table_evaluator
# !pip install ctgan
# !pip install synthcity
# !pip3 install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
! git clone https://github.com/humayraR/fact-ai_Repl.git

Cloning into 'fact-ai_Repl'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 466 (delta 33), reused 14 (delta 5), pack-reused 419
Receiving objects: 100% (466/466), 219.74 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (224/224), done.
Updating files: 100% (133/133), done.


In [4]:
import sys
sys.path.insert(0,'/content/fact-ai_Repl')

In [5]:
%cd fact-ai_Repl

/content/fact-ai_Repl


In [6]:
%pwd

'/content/fact-ai_Repl'

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, recall_score, roc_auc_score

from table_evaluator import TableEvaluator

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

import os.path, sys
from tests.utils import load_adult

import warnings

# synthcity absolute
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [8]:
log.add(sink=sys.stderr, level="INFO")
sys.path.append(os.getcwd())

In [9]:
import PreProcessData

clean_df = PreProcessData.clean_df('data/adult.data')
#arr_X, arr_y, adult_df = load_adult() # arr_X np array without label/target, arr_y np array of just label/target


In [10]:
loader = GenericDataLoader(
    clean_df,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)

In [11]:
# DAG and bias dicts for debiasing adult dataset
df_dag = [
    ('race', 'marital-status'), ('race', 'education'), ('race','occupation'), ('race','hours-per-week'), ('race','label'),

    ('age', 'marital-status'), ('age', 'education'), ('age','occupation'), ('age','hours-per-week'), ('age','workclass'), ('age','relationship'), ('age','label'),

    ('sex', 'marital-status'), ('sex', 'education'), ('sex','occupation'), ('sex','hours-per-week'), ('sex','workclass'), ('sex','relationship'), ('sex','label'),

    ('native-country', 'marital-status'), ('native-country', 'education'), ('native-country','hours-per-week'), ('native-country','workclass'), ('native-country','relationship'), ('native-country','label'),

    ('marital-status', 'education'), ('marital-status','occupation'), ('marital-status','hours-per-week'), ('marital-status','workclass'), ('marital-status','relationship'), ('marital-status','label'),

    ('education','occupation'), ('education','hours-per-week'), ('education','workclass'), ('education','relationship'), ('education','label'),

    ('occupation', 'label'),
    ('hours-per-week', 'label'),
    ('workclass', 'label'),
    ('relationship', 'label')
]

ftu = {'sex':['label']}

dp = {'sex':['label'],
      'marital-status':['label'],
      'relationship':['label'],
      'occupation':['label'],
      'hours-per-week':['label'],
      'workclass':['label'],
      'education':['label']
    }

cf = {'sex':['label'],
      'marital-status':['label'],
      'relationship':['label']
    }

In [9]:
#!pip install nvidia_cublas_cu11 #--yes

In [10]:
#!pip3 install torch torchvision torchaudio

In [12]:
print(torch.version.cuda)

11.7


In [13]:
from synthcity.utils.serialization import load_from_file, save_to_file
from synthcity.utils.reproducibility import clear_cache, enable_reproducible_results
from pathlib import Path
from synthcity.plugins.privacy import plugin_decaf
torch.cuda.empty_cache()
clear_cache()

In [17]:
# synthcity absolute
#from synthcity.plugins import Plugins
# save trained model


workspace = Path("DECAF_workspace")
generator_file = (workspace/"DECAFmodel_32b.bkp")

decaf_model = None
if generator_file.exists():
    decaf_model = load_from_file(generator_file)
else:
    decaf_model = plugin_decaf.DECAFPlugin(n_iter=20, n_iter_baseline=100, batch_size=32)
    decaf_model.fit(loader,dag=df_dag)
    save_to_file(generator_file, decaf_model)


[2023-12-10T22:36:30.209790+0000][7296][INFO] [DECAF] train baseline generator
[2023-12-10T22:36:30.219034+0000][7296][INFO] Encoding race 3843332182083652118
[2023-12-10T22:36:30.225519+0000][7296][INFO] Encoding age 42226137574194105
[2023-12-10T22:36:33.479648+0000][7296][INFO] Encoding sex 7310119158062436415
[2023-12-10T22:36:33.488886+0000][7296][INFO] Encoding native-country 6190545641124984424
[2023-12-10T22:36:37.032646+0000][7296][INFO] Encoding marital-status 1207444760813777779
[2023-12-10T22:36:37.042370+0000][7296][INFO] Encoding education 7923692313984985242
[2023-12-10T22:36:39.540675+0000][7296][INFO] Encoding occupation 2060381146028151082
[2023-12-10T22:36:39.889247+0000][7296][INFO] Encoding hours-per-week 1474711948381798600
[2023-12-10T22:36:43.540067+0000][7296][INFO] Encoding workclass 6611077038566673260
[2023-12-10T22:36:43.549646+0000][7296][INFO] Encoding relationship 163436197888038013
[2023-12-10T22:36:43.559478+0000][7296][INFO] Encoding label 57799050847

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [19]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [12]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:5"

In [22]:
decaf_ftu = decaf_model.generate(count=70000, biased_edges=ftu).dataframe()
decaf_ftu.head()

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,0,42,1,0,0,6,6,49,0,2,1
1,0,45,0,0,2,1,6,35,0,5,1
2,0,26,0,0,2,5,6,34,0,5,1
3,0,17,0,0,2,0,6,31,0,5,1
4,0,59,1,0,0,0,6,35,0,5,1


from matplotlib import pyplot as plt
_df_0['age'].plot(kind='hist', bins=20, title='age')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['sex'].plot(kind='hist', bins=20, title='sex')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['marital-status'].plot(kind='hist', bins=20, title='marital-status')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['education'].plot(kind='hist', bins=20, title='education')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['education']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_27.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('education')

from matplotlib import pyplot as plt
_df_28['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_29['age'].plot(kind='line', figsize=(8, 4), title='age')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_30['sex'].plot(kind='line', figsize=(8, 4), title='sex')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_31['marital-status'].plot(kind='line', figsize=(8, 4), title='marital-status')
plt.gca().spines[['top', 'right']].set_visible(False)

In [26]:
decaf_ftu['label'].unique()

array([1])

In [29]:
def get_metrics(mode,df,synthetic):

    # Split the data into train,test
    traindf, testdf = train_test_split(df, test_size=0.3)
    X_train = traindf.loc[:, traindf.columns != 'label']
    y_train = traindf['label']
    X_test = testdf.loc[:, testdf.columns != 'label']
    y_test = testdf['label']

    clf_df = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',
                                     learning_rate='constant', learning_rate_init=0.001).fit(X_train, y_train)
    '''
    SYNTHETIC DATASET
    '''
    # Make sure the data is representative of the original dataset
    synthetic_balanced_1 = synthetic[synthetic.label == 1].sample(frac = 0.75)
    synthetic_balanced_0 = synthetic[synthetic.label == 0].sample(frac = 0.25)
    #synthetic_balanced = synthetic_balanced_1.append(synthetic_balanced_0)
    synthetic_balanced = pd.concat([synthetic_balanced_1, synthetic_balanced_0], ignore_index=True)

    # Split the data into train,test
    X_syn = synthetic_balanced.loc[:, synthetic_balanced.columns != 'label']
    y_syn = synthetic_balanced['label']

    y_pred_syn = clf_df.predict(X_syn)

    synthetic_pos = synthetic.assign(sex=0)
    synthetic_neg = synthetic.assign(sex=1)

    x_pos_syn = synthetic_balanced[synthetic_balanced['sex'] == 0].drop(['label'], axis = 1)[:7508]
    x_neg_syn = synthetic_balanced[synthetic_balanced['sex'] == 1].drop(['label'], axis = 1)[:7508]

    pos = clf_df.predict(synthetic_pos.drop('label',axis=1))
    neg = clf_df.predict(synthetic_neg.drop('label',axis=1))

    pred_pos_syn = clf_df.predict(x_pos_syn)
    pred_neg_syn = clf_df.predict(x_neg_syn)

    FTU = np.abs(np.mean(pos-neg))
    DP = np.mean(pred_pos_syn)-np.mean(pred_neg_syn)

    # Print the obtained statistics
    print('Statistics for dataset for mode:',mode)
    print('Precision:',precision_score(y_syn, y_pred_syn, average='binary'))
    print('Recall:',recall_score(y_syn, y_pred_syn, average='binary'))
    print('AUROC:',roc_auc_score(y_syn, y_pred_syn))
    print('FTU:',FTU)
    print('DP:',DP)

get_metrics("DECAF-FTU", decaf_ftu,clean_df)

Statistics for dataset for mode: DECAF-FTU
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


In [30]:
decaf_dp = decaf_model.generate(count=70000, biased_edges=dp).dataframe()
decaf_dp.head()

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,0,55,1,0,0,0,6,49,0,5,1
1,0,35,1,0,0,4,6,34,0,5,1
2,0,20,1,0,2,5,6,34,0,3,1
3,0,43,0,0,1,5,6,22,0,5,1
4,0,49,1,0,0,2,6,49,0,5,1


In [33]:
decaf_dp['relationship'].unique()

array([5, 3, 1, 2])

In [34]:
get_metrics("DECAF-DP", decaf_dp, clean_df)

Statistics for dataset for mode: DECAF-DP
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


In [35]:
decaf_cf = decaf_model.generate(count=70000, biased_edges=cf).dataframe()
decaf_cf.head()

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,0,53,1,20,2,2,6,13,0,5,1
1,0,47,1,0,0,6,6,49,0,5,1
2,0,22,0,0,0,1,6,35,0,5,1
3,0,29,1,0,0,5,6,49,0,5,1
4,0,29,1,0,0,3,6,35,0,5,1


In [36]:
decaf_cf['relationship'].unique()

array([5, 1, 3, 2])

In [37]:
get_metrics("DECAF-cf", decaf_cf, clean_df)

Statistics for dataset for mode: DECAF-cf
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


In [38]:
#save csv

decaf_ftu.to_csv('decaf_ftu.csv', index=False)
decaf_dp.to_csv('decaf_dp.csv', index=False)
decaf_cf.to_csv('decaf_cf.csv', index=False)


In [39]:
#get metrics for real data
get_metrics("Original Data",clean_df,clean_df)

Statistics for dataset for mode: Original Data
Precision: 0.9514575023013194
Recall: 0.9125367863449088
AUROC: 0.7455598156551395
FTU: 0.09087593660897818
DP: 0.08608662141188894


In [ ]:
# Synthcity Benchmarking doesn't support DECAF at the moment. Can't pass in DAG or bias dict. Will need to build benchmarking from scratch for DECAF and other external GANs
# from SynthBenchmarks import Benchmarks

# score = Benchmarks.evaluate(
#     [("Test Decaf", "decaf", {"n_iter": 10, "n_iter_baseline": 100})],
#     loader,
#     synthetic_size=1000,
#     repeats=2
#     #synthetic_cache=False
#     #synthetic_reuse_if_exists=True
# )

In [ ]:
clean_orig_df = PreProcessData.clean_df('data/adult.data')
traindf_orig_df, test_orig_df = train_test_split(clean_orig_df, test_size=0.95)

ftu_loader = GenericDataLoader(
    decaf_ftu,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)


# train and test DPGAN on FACTGAN synthetic data
#clean_df = PreProcessData.clean_df('data/adult.data')
test_loader = GenericDataLoader(
    test_orig_df,
    target_column="label",
    sensitive_columns=["race","sex","native-country"],
)

In [ ]:
from synthcity.plugins.privacy import plugin_dpgan

ftu_dpgan_e01 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=0.1)
ftu_dpgan_e01.fit(ftu_loader)

[2023-12-07T01:19:33.394382-0600][17660][INFO] Encoding race 4064610612366098312
[2023-12-07T01:19:33.432704-0600][17660][INFO] Encoding age 3525822842710635444
[2023-12-07T01:19:35.576969-0600][17660][INFO] Encoding sex 2286248976586486390
[2023-12-07T01:19:35.576969-0600][17660][INFO] Encoding native-country 6523902198192136605
[2023-12-07T01:19:36.028232-0600][17660][INFO] Encoding marital-status 4735422621724849635
[2023-12-07T01:19:36.028232-0600][17660][INFO] Encoding education 8650874399102879509
[2023-12-07T01:19:36.774015-0600][17660][INFO] Encoding occupation 19303803562259629
[2023-12-07T01:19:36.774015-0600][17660][INFO] Encoding hours-per-week 3513447030895649126
[2023-12-07T01:19:36.789601-0600][17660][INFO] Encoding workclass 3206741733929796058
[2023-12-07T01:19:36.800789-0600][17660][INFO] Encoding relationship 3206741733929796058
[2023-12-07T01:19:36.800789-0600][17660][INFO] Encoding label 3206741733929796058
[2023-12-07T01:19:37.981185-0600][17660][INFO] Training GA

In [ ]:
ftu_dpgan_e01_df = ftu_dpgan_e01.generate(count=50000).dataframe()
ftu_dpgan_e01_df.head()

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,1,79,0,14,3,7,3,17,4,1,1
1,4,77,0,14,3,7,3,17,4,1,1
2,1,75,0,13,3,7,3,17,4,1,1
3,1,71,0,13,3,6,4,23,4,1,1
4,1,79,0,7,6,7,3,17,4,1,1


In [ ]:
get_metrics("ftu_dpgan_e01", ftu_dpgan_e01_df, clean_df)

Statistics for dataset for mode: ftu_dpgan_e01
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


In [ ]:
from synthcity.benchmark import Benchmarks

ftu_dpgan_e01_score2 = Benchmarks.evaluate(
    [("dp_decaf_ftu_eps_0.1", "dpgan", {"epsilon": 0.1, "n_iter": 10, "n_iter_min": 1})],
    ftu_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(ftu_dpgan_e01_score2)

[2023-12-07T02:00:54.851415-0600][17660][INFO] Testcase : dp_decaf_ftu_eps_0.1
[2023-12-07T02:00:54.851415-0600][17660][INFO] [testcase] Experiment repeat: 0 task type: classification Train df hash = 8902828986042714684
[2023-12-07T02:01:12.546642-0600][17660][INFO] Encoding race 5929786346541741767
[2023-12-07T02:01:12.552642-0600][17660][INFO] Encoding age 8387398801735511711
[2023-12-07T02:01:14.755126-0600][17660][INFO] Encoding sex 7824932933582357646
[2023-12-07T02:01:14.755126-0600][17660][INFO] Encoding native-country 7171068313903795367
[2023-12-07T02:01:15.172860-0600][17660][INFO] Encoding marital-status 8018768697944197846
[2023-12-07T02:01:15.183858-0600][17660][INFO] Encoding education 8775285848867082620
[2023-12-07T02:01:15.556705-0600][17660][INFO] Encoding occupation 4359471017579139326
[2023-12-07T02:01:15.572363-0600][17660][INFO] Encoding hours-per-week 5826843159508513143
[2023-12-07T02:01:15.572363-0600][17660][INFO] Encoding workclass 6987581915405272524
[2023-1


Plugin : dp_decaf_ftu_eps_0.1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
privacy.delta-presence.score,17.600000,17.600000,17.600000,0.0,17.600000,0.0,1,0,0.82
privacy.identifiability_score.score,0.001202,0.001202,0.001202,0.0,0.001202,0.0,1,0,29.47
privacy.identifiability_score.score_OC,0.008786,0.008786,0.008786,0.0,0.008786,0.0,1,0,29.47


In [ ]:
ftu_dpgan_e1 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=1)
ftu_dpgan_e1.fit(ftu_loader)
ftu_dpgan_e1_df = ftu_dpgan_e1.generate(count=50000).dataframe()
ftu_dpgan_e1_df.head()

[2023-12-07T02:19:06.812055-0600][17660][INFO] Encoding race 4064610612366098312
[2023-12-07T02:19:06.812055-0600][17660][INFO] Encoding age 3525822842710635444
[2023-12-07T02:19:08.797667-0600][17660][INFO] Encoding sex 2286248976586486390
[2023-12-07T02:19:08.797667-0600][17660][INFO] Encoding native-country 6523902198192136605
[2023-12-07T02:19:09.231831-0600][17660][INFO] Encoding marital-status 4735422621724849635
[2023-12-07T02:19:09.247456-0600][17660][INFO] Encoding education 8650874399102879509
[2023-12-07T02:19:09.847253-0600][17660][INFO] Encoding occupation 19303803562259629
[2023-12-07T02:19:09.847253-0600][17660][INFO] Encoding hours-per-week 3513447030895649126
[2023-12-07T02:19:09.862844-0600][17660][INFO] Encoding workclass 3206741733929796058
[2023-12-07T02:19:09.873915-0600][17660][INFO] Encoding relationship 3206741733929796058
[2023-12-07T02:19:09.881863-0600][17660][INFO] Encoding label 3206741733929796058
[2023-12-07T02:19:10.955570-0600][17660][INFO] Training GA

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,3,32,1,6,6,6,3,21,4,1,1
1,3,32,0,0,6,6,3,21,4,1,1
2,3,32,1,6,6,7,3,21,4,1,1
3,3,32,0,6,6,7,3,23,4,1,1
4,3,32,0,6,6,6,3,21,4,1,1


In [ ]:
get_metrics("ftu_dpgan_e1", ftu_dpgan_e1_df, clean_df)
ftu_dpgan_e1_score2 = Benchmarks.evaluate(
    [("dp_decaf_ftu_eps_1", "dpgan", {"epsilon": 1, "n_iter": 10, "n_iter_min": 1})],
    ftu_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(ftu_dpgan_e1_score2)

[2023-12-07T02:39:42.045383-0600][17660][INFO] Testcase : dp_decaf_ftu_eps_1
[2023-12-07T02:39:42.048353-0600][17660][INFO] [testcase] Experiment repeat: 0 task type: classification Train df hash = 8902828986042714684
[2023-12-07T02:39:42.109089-0600][17660][INFO] Encoding race 5929786346541741767
[2023-12-07T02:39:42.114088-0600][17660][INFO] Encoding age 8387398801735511711


Statistics for dataset for mode: ftu_dpgan_e1
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-07T02:39:43.623591-0600][17660][INFO] Encoding sex 7824932933582357646
[2023-12-07T02:39:43.628623-0600][17660][INFO] Encoding native-country 7171068313903795367
[2023-12-07T02:39:43.970984-0600][17660][INFO] Encoding marital-status 8018768697944197846
[2023-12-07T02:39:43.975986-0600][17660][INFO] Encoding education 8775285848867082620
[2023-12-07T02:39:44.301770-0600][17660][INFO] Encoding occupation 4359471017579139326
[2023-12-07T02:39:44.306795-0600][17660][INFO] Encoding hours-per-week 5826843159508513143
[2023-12-07T02:39:44.312793-0600][17660][INFO] Encoding workclass 6987581915405272524
[2023-12-07T02:39:44.317930-0600][17660][INFO] Encoding relationship 6987581915405272524
[2023-12-07T02:39:44.323900-0600][17660][INFO] Encoding label 6987581915405272524
[2023-12-07T02:39:45.077162-0600][17660][INFO] Training GAN on device cpu. features = 45
100%|██████████| 10/10 [15:33<00:00, 93.39s/it]
[2023-12-07T02:55:38.929620-0600][17660][ERROR]  >> Evaluator privacy.DomiasMIA_


Plugin : dp_decaf_ftu_eps_1


,min,max,mean,stddev,median,iqr,rounds,errors,durations
privacy.delta-presence.score,1.710817,1.710817,1.710817,0.0,1.710817,0.0,1,0,0.84
privacy.identifiability_score.score,0.001616,0.001616,0.001616,0.0,0.001616,0.0,1,0,2.09
privacy.identifiability_score.score_OC,0.012806,0.012806,0.012806,0.0,0.012806,0.0,1,0,2.09


In [ ]:
ftu_dpgan_e10 = plugin_dpgan.DPGANPlugin(n_iter = 10, n_iter_min=1, epsilon=10)
ftu_dpgan_e10.fit(ftu_loader)
ftu_dpgan_e10_df = ftu_dpgan_e10.generate(count=50000).dataframe()
ftu_dpgan_e10_df.head()

[2023-12-07T02:55:39.068115-0600][17660][INFO] Encoding race 4064610612366098312
[2023-12-07T02:55:39.073116-0600][17660][INFO] Encoding age 3525822842710635444
[2023-12-07T02:55:40.874779-0600][17660][INFO] Encoding sex 2286248976586486390
[2023-12-07T02:55:40.879951-0600][17660][INFO] Encoding native-country 6523902198192136605
[2023-12-07T02:55:41.257729-0600][17660][INFO] Encoding marital-status 4735422621724849635
[2023-12-07T02:55:41.263456-0600][17660][INFO] Encoding education 8650874399102879509
[2023-12-07T02:55:41.799397-0600][17660][INFO] Encoding occupation 19303803562259629
[2023-12-07T02:55:41.805400-0600][17660][INFO] Encoding hours-per-week 3513447030895649126
[2023-12-07T02:55:41.812537-0600][17660][INFO] Encoding workclass 3206741733929796058
[2023-12-07T02:55:41.817538-0600][17660][INFO] Encoding relationship 3206741733929796058
[2023-12-07T02:55:41.822569-0600][17660][INFO] Encoding label 3206741733929796058
[2023-12-07T02:55:42.858785-0600][17660][INFO] Training GA

,race,age,sex,native-country,marital-status,education,occupation,hours-per-week,workclass,relationship,label
0,1,31,1,21,1,6,3,22,4,1,1
1,1,17,1,6,1,7,3,21,4,1,1
2,1,64,1,39,5,11,4,21,4,1,1
3,1,52,1,39,6,7,4,23,4,1,1
4,3,35,0,6,1,2,3,21,4,1,1


In [ ]:
get_metrics("ftu_dpgan_e10", ftu_dpgan_e10_df, clean_df)
ftu_dpgan_e10_score2 = Benchmarks.evaluate(
    [("dp_decaf_ftu_eps_10", "dpgan", {"epsilon": 10, "n_iter": 10, "n_iter_min": 1})],
    ftu_loader,
    test_loader,
    synthetic_size=50000,
    repeats=1,
    metrics={
        'privacy': ['delta-presence', 'identifiability_score', 'DomiasMIA_prior']
    },
    # use_metric_cache=False
)
Benchmarks.print(ftu_dpgan_e10_score2)

[2023-12-07T03:15:04.270598-0600][17660][INFO] Testcase : dp_decaf_ftu_eps_10
[2023-12-07T03:15:04.273741-0600][17660][INFO] [testcase] Experiment repeat: 0 task type: classification Train df hash = 8902828986042714684
[2023-12-07T03:15:04.324994-0600][17660][INFO] Encoding race 5929786346541741767
[2023-12-07T03:15:04.330120-0600][17660][INFO] Encoding age 8387398801735511711


Statistics for dataset for mode: ftu_dpgan_e10
Precision: 0.9005141251921344
Recall: 1.0
AUROC: 0.5
FTU: 0.0
DP: 0.0


[2023-12-07T03:15:05.782274-0600][17660][INFO] Encoding sex 7824932933582357646
[2023-12-07T03:15:05.787275-0600][17660][INFO] Encoding native-country 7171068313903795367
[2023-12-07T03:15:06.069523-0600][17660][INFO] Encoding marital-status 8018768697944197846
[2023-12-07T03:15:06.074524-0600][17660][INFO] Encoding education 8775285848867082620
[2023-12-07T03:15:06.368104-0600][17660][INFO] Encoding occupation 4359471017579139326
[2023-12-07T03:15:06.373226-0600][17660][INFO] Encoding hours-per-week 5826843159508513143
[2023-12-07T03:15:06.378226-0600][17660][INFO] Encoding workclass 6987581915405272524
[2023-12-07T03:15:06.383227-0600][17660][INFO] Encoding relationship 6987581915405272524
[2023-12-07T03:15:06.388567-0600][17660][INFO] Encoding label 6987581915405272524
[2023-12-07T03:15:07.149908-0600][17660][INFO] Training GAN on device cpu. features = 45
100%|██████████| 10/10 [15:17<00:00, 91.78s/it]
[2023-12-07T03:34:47.053892-0600][17660][ERROR]  >> Evaluator privacy.DomiasMIA_


Plugin : dp_decaf_ftu_eps_10


,min,max,mean,stddev,median,iqr,rounds,errors,durations
privacy.delta-presence.score,14.351852,14.351852,14.351852,0.0,14.351852,0.0,1,0,0.74
privacy.identifiability_score.score,0.001326,0.001326,0.001326,0.0,0.001326,0.0,1,0,2.02
privacy.identifiability_score.score_OC,0.010278,0.010278,0.010278,0.0,0.010278,0.0,1,0,2.02
